In [48]:
import torch
import cv2 # OpenCV-Python is a library of Python bindings designed to solve computer vision problems / displays an image 
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from sklearn import neighbors

In [49]:
import sys
sys.executable

'C:\\Users\\Hset Hset Naing\\anaconda3\\envs\\TSM\\python.exe'

In [50]:
def draw_ellipse(canvas,a,b):
    #draw a random ellipse

    h,w = canvas.shape[0], canvas.shape[1]
    center_y = int((h-1)/2)  #np.random.randint(-h/4,h/4) + int(h/2)
    center_x = int((w-1)/2) #np.random.randint(-w/4,w/4) + int(w/2)
    axis1 = np.random.randint(a/2,a)
    axis2 = np.random.randint(b/2,b)
    angle = np.random.rand() * 90

    return cv2.ellipse(canvas,(center_x,center_y),(axis1,axis2),angle, 0,360,(1,1,1),thickness=-1)

In [47]:
# test 
pts = np.array([[1,1], [2,4], [5,3]])
    
pts = pts - np.array([10,10])

print("pts", pts)

a = np.array([[1,2,3,8], [4,5,6,7]])
print("reshape", np.reshape(a, (4,-1)))

pts [[-9 -9]
 [-8 -6]
 [-5 -7]]
reshape [[1 2]
 [3 8]
 [4 5]
 [6 7]]


In [51]:
def draw_polygon(canvas,low_num_pts,high_num_pts):
    #draw a random polygon whose number of sides is between low_num and high_num
    
    # image height and width determined by parameter 
    img_height = canvas.shape[0]
    img_width = canvas.shape[1]
    
    # parameters are low and high num, num_pts is a random point between these lower and upper bounds
    # high_num + 1 to include it
    num_pts = np.random.randint(low = low_num_pts,high = high_num_pts+1) 
    
    # size refers to size and shape of array, num_pts: no. of arrays, 2: no. of el in each array 
    # no width needed? 
    # points are represented with 2 values (x,y) in 2d space. pts are a list of all these randomly generated points
    pts = np.random.randint(low = (img_height * 0.25), high = (img_height * 0.75),size = [num_pts,2])
    
    # not rly sure why subtracting 10?
    pts = pts - np.array([10,10])
    
    # cv2.fillpoly(Image,End_Points,Color) 
    # np.zeros -> blank canvas? array filled with zeroes
    # (1,1,1) refers to color 
    
    # image is binary (2 colors)  
    binary = cv2.fillConvexPoly(np.zeros(canvas.shape),pts,(1,1,1))
    
    # "A blob is a group of connected pixels in an image that shares some common property"
    # "Image moment is a particular weighted average of image pixel intensities"
    M = cv2.moments(binary)
    
    
    # calculate x,y coordinate of center
    # adjusted from usual calc by having x-1/2
    # not sure purpose of x-1/2?
    cX = int((img_width-1)/2 - M["m10"] / M["m00"] )
    cY = int((img_height-1)/2 -M["m01"] / M["m00"] )
    center = np.array([cX,cY])

    # opposite of prev line (subtract 10), this one adds centre values 
    pts = pts + center

    # draw a filled polygon 
    # recentered bc pts had center added 
    recentered = cv2.fillConvexPoly(canvas,pts,(1,1,1))

    return recentered

In [52]:
def get_rand_vec(num_vecs):
    #generates a random vector
    
    # num_vecs refers to no. of arrays, 2 refers to no. of elements in each array 
    vec = np.random.rand(num_vecs,2)
    
    # axis refers to which way you add the vectors 
    magnitudes = np.sqrt(np.sum(vec**2,axis = 1))
    
    # reshape is reordering np array 
    vec = vec / np.reshape(magnitudes,[num_vecs,1])

    return vec


In [53]:
def find_vec_len(img,vecs,center,max_iter):
    #extends out one at a time, check if the pixel
    #vec must have magnitude 1

    to_ret = []
    for v_num in range(vecs.shape[0]):
        for i in range(max_iter):
            
            # i refers to number of steps taken in the direction of vector
            # as represented by vecs[v_num]
            pos_to_check = np.round(center + i * vecs[v_num])
            
            # check if value of image at the position is < 1
            # if there is nothing there, append it to list and return
            # this keeps track of no. of "steps" taken from center, which
            # corresponds to the length of the vector 
            if img[int(pos_to_check[0]),int(pos_to_check[1])] < 1:
                to_ret.append(i-1)
                break

    return np.array(to_ret)

In [14]:
def get_length_encoding(canvass,num_samples,bins):
    #get the length encoding of a binary image

    num_drawings = canvass.shape[0]
    canvas_width = canvass.shape[2]
    canvas_height= canvass.shape[1]

    center = np.array([(canvas_height-1) / 2, (canvas_width-1) / 2])
    lengthss = []

    vecs = get_rand_vec(num_samples)

    for i in range(num_drawings):
        lengths = find_vec_len(canvass[i],vecs,center,int(1.41*(canvas_width-1)/2))
        hist,bin_edges = np.histogram(lengths,bins = bins,density=True)
        lengthss.append(hist)

    lengthss = np.array(lengthss)

    return lengthss

In [15]:
def imgs_to_adj_mat(imgs):
    #convert binary image to adjacency matrix
    num_imgs = imgs.shape[0]
    to_ret = []
    for img_idx in range(num_imgs):
        nonzero_x, nonzero_y = np.nonzero(imgs[img_idx])
        coords = np.stack([nonzero_x, nonzero_y]).T
        adj_mat = neighbors.radius_neighbors_graph(coords, np.sqrt(2))

        to_ret.append(adj_mat)

    return to_ret

In [16]:
def adj_mat_to_lap_mat(adj_mats):
    #inputs a list of adjacency matrices, outputs a list of laplacian matrices
    num_mats = len(adj_mats)
    to_ret = []

    for mat_idx in range(num_mats):
        adj_mat = adj_mats[mat_idx]
        diag_mat = np.diag(np.sum(adj_mat,axis = 0))
        lap_mat = diag_mat - adj_mat
        #gr = nx.from_scipy_sparse_array(adj_mat)
        #lap_mat = nx.laplacian_matrix(gr).toarray()
        to_ret.append(lap_mat)
    return to_ret

In [17]:
def lap_mat_to_eigen(lap_mats):
    #compute the eigenvalues and eigenvectors of a list of laplacian matrices
    num_mats = len(lap_mats)

    evals = []
    Us = []

    for mat_idx in range(num_mats):
        lap_mat = lap_mats[mat_idx]
        eval, U = np.linalg.eigh(lap_mat)
        evals.append(eval)
        Us.append(U)

    return evals, Us

In [18]:
def heat_kernel_t(t, evals, U):
    #find the heat kernel at time t
    exp = np.exp(t * evals)
    lamb = np.diag(exp)
    return U.dot(lamb).dot(U.T)

In [19]:
def heat_kernel_signature(ts, eval, U):
    #find the heat kernel signature given a bunch of ts
    toret = np.empty([ts.shape[0]])
    for i in range(ts.shape[0]):
        t = ts[i]
        kernel = heat_kernel_t(t, eval, U)
        diagonal = np.diag(kernel)
        toret[i] = np.mean(diagonal)

    return toret

In [20]:
def batch_heat_kernel_signature(ts,evals,Us):
    #find heat kernel signature given a bunch of ts of a bunch of matrices
    num_mats = len(evals)
    to_ret = np.empty((num_mats,ts.shape[0]))

    for mat_idx in range(num_mats):
        hks = heat_kernel_signature(ts,evals[mat_idx],Us[mat_idx])
        to_ret[mat_idx] = hks

    return to_ret